In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

In [2]:
df1 = pd.read_csv('../data/feature_dispositivo_mas_usado.csv', low_memory=False, index_col='person')
df2 = pd.read_csv('../data/feature_cantidad_de_eventos_en_intervalos.csv', low_memory=False, index_col='person')
df3 = pd.read_csv('../data/feature_dispositivos_storage.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('../data/features_basicas_ult_quin.csv', low_memory=False, index_col='person')
df5 = pd.read_csv('../data/features_ultima_ocurrencia_cada_evento.csv', low_memory=False, index_col='person')
df6 = pd.read_csv('../data/days_elapsed_from_last_event.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('../data/featureUsuarioRealiza30EventosEn20MinsLosUltimos2Dias.csv', low_memory=False, index_col='person')
df8 = pd.read_csv('../data/features_ultimo_checkout.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('../data/diferencia_de_count_de_eventos.csv', low_memory=False, index_col='person')
df10 = pd.read_csv('../data/top_10_celulares.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('../data/feature_cantidad_de_eventos_en_intervalos_ult_quin.csv', low_memory=False, index_col='person')
df12 = pd.read_csv('../data/featureUsuariosRegistraronActividadEl293031YUltimaHora.csv', low_memory=False, index_col='person')
df13 = pd.read_csv('../data/cant_max_eventos_sobre_un_mismo_telefono_ult_quincena.csv', low_memory=False, index_col='person')
df14 = pd.read_csv('../data/feature_color_ult15dias.csv', low_memory=False, index_col='person')

labels = pd.read_csv('../data/labels_training_set.csv', low_memory=False, index_col='person')

df_unidos = df1.join(df2, how='inner')
df_unidos = df_unidos.join(df3, how='inner')
df_unidos = df_unidos.join(df4, how='inner')
df_unidos = df_unidos.join(df5, how='inner')
df_unidos = df_unidos.join(df6, how='inner')
df_unidos = df_unidos.join(df7, how='inner')
df_unidos = df_unidos.join(df8, how='inner')
df_unidos = df_unidos.join(df9).fillna(0)
df_unidos = df_unidos.join(df10, how='inner')
df_unidos = df_unidos.join(df11, how='inner')
df_unidos = df_unidos.join(df12, how='inner')
df_unidos = df_unidos.join(df13, how='inner')
df_unidos = df_unidos.join(df14, how='inner')
#  Le asigno las features al subset de usuarios que nos da Trocafone para entrenar.
df_test = df_unidos.join(labels, on='person', how='inner')
df_test.shape

(19414, 66)

In [3]:
features = df_test.columns.tolist()
features.remove('label')

In [4]:
X = df_test[features]
Y = df_test['label']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=311)

In [7]:
param_grid = { 
    'n_estimators': [x for x in range(1000, 2000, 200)],
    'max_depth': [7,10, 12],
    'num_leaves' : [70, 512, 2048],
    'learning_rate': [0.1, 0.01],
    'min_data_in_leaf': [100, 500]
}

In [8]:
_lgb = lgb.LGBMClassifier(random_state=80)
CV_lgb = GridSearchCV(estimator=_lgb, param_grid=param_grid, cv= 5, scoring='roc_auc', n_jobs=4)
CV_lgb.fit(X_train, y_train)

/home/alawichu/.local/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=80, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'n_estimators': [1000, 1200, 1400, 1600, 1800], 'max_depth': [7, 10, 12], 'num_leaves': [70, 512, 2048], 'learning_rate': [0.1, 0.01], 'min_data_in_leaf': [100, 500]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [9]:
CV_lgb.best_params_

{'learning_rate': 0.01,
 'max_depth': 7,
 'min_data_in_leaf': 500,
 'n_estimators': 1000,
 'num_leaves': 70}

In [10]:
CV_lgb.best_score_

0.8700749686212016

In [16]:
y_pred = CV_lgb.predict_proba(X_test)
y_pred_proba = [p[1] for p in y_pred]
print(roc_auc_score(y_test, y_pred_proba))

0.8540469649398221


In [11]:
df_submit = pd.read_csv('../data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_unidos[features], how='inner')

In [12]:
df_events.shape

(19415, 65)

In [13]:
kaggle_pred = CV_lgb.predict_proba(df_events)
proba_de_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_de_comprar)
df_submit['label'] = series.values

In [14]:
df_submit.to_csv('../submit_lgbm_v6_features_15dias.csv')